<a href="https://colab.research.google.com/github/diptisahu/Emoji-Prediction/blob/main/Lstm-GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

import re
import pandas as pd
import numpy as np

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2022-03-16 00:33:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-16 00:33:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-16 00:33:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [3]:
!unzip glove*.zip


Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [4]:
glove_dir = './glove.6B.200d.txt'
input_dim = 200

vocab = {}
with open(glove_dir, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        vocab[word] = vector
f.close()

print('Found %s word vectors.' %len(vocab))

Found 400000 word vectors.


In [5]:
max_len = 50
zero_padding = [0]*input_dim

def get_embeddings(text, emb="LSTM"):
    if emb == "DNN":
        embedding = [0]*input_dim
        i = 0
        for word in text.split(' '):
            if word in vocab:
                i += 1
                embedding += vocab[word]

        if i != 0:
            embedding /= i
    elif emb == "LSTM":
        embedding = []
        i = 0
        for word in text.split(' '):
            if i == max_len:
                break
            if word in vocab:
                i += 1
                scale = 0.1
                noise = np.random.randn(input_dim)*scale
                embedding.append(vocab[word] + noise)
        
        while i < max_len:
            i += 1
            embedding.append(zero_padding)
            
    return embedding

In [6]:
def clean(text):
    """ Function to clean the text """
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    
    # Remove numbers from string
    texter = re.sub(pattern=r"[+-]?\d+(?:\.\d+)?", repl="", string=texter, count=0, flags=0)
    texter = texter.replace("  ", " ")
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    texter = re.sub(r'[^\w\s]', '', texter)
    if texter == "":
        texter = ""
    return texter

In [7]:
def transform(X, emb="LSTM"):
    embeddings = []
    for item in X:
        item = clean(item)
        embedding = get_embeddings(item, emb)
        embeddings.append(embedding)
    
    return embeddings

In [8]:
inputs_file = "/content/tweets_training.txt"
labels_file = "/content/tweets_training_labels.txt"

X = []
with open(inputs_file, encoding='utf8') as f:
    for line in f:
        X.append(line)
f.close()
print(len(X))

y = []
with open(labels_file, encoding='utf8') as f:
    for line in f:
        y.append(int(line))
f.close()

352867


In [9]:
# Hyperparameters
num_epochs = 3
batch_size = 100
learning_rate = 0.0005
dropout = 0.5

In [10]:
## LSTM

with torch.no_grad():
  torch.cuda.empty_cache()
  X = transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

  X_tr = torch.tensor(X_train, dtype=torch.float).to('cuda')
  y_tr = torch.tensor(y_train).to('cuda')
  train = TensorDataset(X_tr, y_tr)
  trainloader = DataLoader(train, batch_size=batch_size)
  torch.cuda.empty_cache()

  X_te = torch.tensor(X_test, dtype=torch.float).to('cuda')
  y_te = torch.tensor(y_test).to('cuda')
  test = TensorDataset(X_te, y_te)
  testloader = DataLoader(test, batch_size=batch_size)

  print(len(y_tr))
  print(X_tr.size())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


299936
torch.Size([299936, 50, 200])


In [11]:
class LSTM(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True).to('cuda') #lstm
        self.fc1 =  nn.Linear(hidden_size*seq_length, 128) #fully connected 1
        self.fc2 = nn.Linear(128, num_classes) #fully connected last layer
        
        self.dropout = nn.Dropout(dropout)
        self.batchnorm1 = nn.BatchNorm1d(128)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to('cuda') #hidden state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to('cuda')
        
        # Propagate input through LSTM
        x, _ = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        # Flatten lstm output
        x = x.reshape(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
#         x = F.log_softmax(self.fc2(x), dim=1)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

lstm = LSTM(20, 200, 64, 3, 50, dropout).to('cuda')

print(lstm)

LSTM(
  (lstm): LSTM(200, 64, num_layers=3, batch_first=True, dropout=0.5)
  (fc1): Linear(in_features=3200, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [12]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

In [13]:
# Training
for epoch in range(num_epochs):
    running_loss = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = inputs.to('cuda')
        labels= labels.to('cuda')
        outputs = lstm(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

        running_loss += loss.item()
        
        if (i+1) % 250 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(trainloader), loss.data))

Epoch [1/3], Step [250/3000], Loss: 2.5790
Epoch [1/3], Step [500/3000], Loss: 2.3972
Epoch [1/3], Step [750/3000], Loss: 2.3425
Epoch [1/3], Step [1000/3000], Loss: 2.4824
Epoch [1/3], Step [1250/3000], Loss: 2.2709
Epoch [1/3], Step [1500/3000], Loss: 2.4831
Epoch [1/3], Step [1750/3000], Loss: 2.5753
Epoch [1/3], Step [2000/3000], Loss: 2.1950
Epoch [1/3], Step [2250/3000], Loss: 2.3325
Epoch [1/3], Step [2500/3000], Loss: 2.4319
Epoch [1/3], Step [2750/3000], Loss: 2.3682
Epoch [1/3], Step [3000/3000], Loss: 2.2671
Epoch [2/3], Step [250/3000], Loss: 2.3114
Epoch [2/3], Step [500/3000], Loss: 2.2829
Epoch [2/3], Step [750/3000], Loss: 2.1968
Epoch [2/3], Step [1000/3000], Loss: 2.4571
Epoch [2/3], Step [1250/3000], Loss: 2.1748
Epoch [2/3], Step [1500/3000], Loss: 2.4073
Epoch [2/3], Step [1750/3000], Loss: 2.4273
Epoch [2/3], Step [2000/3000], Loss: 2.0916
Epoch [2/3], Step [2250/3000], Loss: 2.3059
Epoch [2/3], Step [2500/3000], Loss: 2.3732
Epoch [2/3], Step [2750/3000], Loss: 2

In [14]:
# Testing
lstm.eval()
total, correct = 0, 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data

        outputs = lstm(inputs)

        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum()

print(f'Accuracy of the model is: {100*correct/total:.2f}%')

Accuracy of the model is: 32.16%
